# 03 - Benchmark Completo

Este notebook executa o benchmark completo comparando múltiplos modelos LLM.

## Objetivos
1. Testar todos os modelos configurados
2. Comparar todas as estratégias de prompting
3. Avaliar em múltiplos datasets
4. Salvar resultados em arquivos JSON

⚠️ **Atenção**: Este notebook pode levar várias horas para executar!

📊 **Análise**: Após a execução, use o notebook `04_benchmark_analysis.ipynb` para analisar os resultados.

In [ ]:
# Configurar path do projeto
import sys
from pathlib import Path
from loguru import logger

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root / "src"))

# Configurar logger
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add(project_root / "reports/logs/03_full_benchmark.log", rotation="10 MB")

logger.info(f"Project root: {project_root}")

In [ ]:
# Imports
import json
import time
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm

# Módulos do projeto
from config import MODELS, PROMPTING_STRATEGIES, DATASETS, EXPERIMENT_CONFIG, PATHS
from data.data_loader import load_dataset, prepare_test_set
from models.model_handler import ModelHandlerOllama
from models.prompts import PromptBuilder
from models.metrics import calculate_metrics, generate_report

# Verificar Ollama
try:
    import ollama
    ollama_models = ollama.list()
    # Ollama retorna ListResponse com .models (não dict com 'models')
    models_list = ollama_models.models if hasattr(ollama_models, 'models') else []
    logger.info(f"✅ Ollama conectado. Modelos disponíveis: {len(models_list)}")
    for model in models_list:
        # Cada modelo é um objeto com atributo .model (não dict com 'name')
        model_name = model.model if hasattr(model, 'model') else str(model)
        logger.info(f"   - {model_name}")
except Exception as e:
    logger.error(f"❌ Erro ao conectar com Ollama: {e}")
    logger.error("   Certifique-se de que Ollama está instalado e rodando.")

## 1. Configuração do Benchmark

In [ ]:
# Configurações
SAMPLE_SIZE = 1000  # Número de amostras por dataset
SEED = 42

# Modelos a testar (pode reduzir para teste rápido)
MODELS_TO_TEST = MODELS  # Descomentar para testar todos

# Estratégias a testar
STRATEGIES_TO_TEST = PROMPTING_STRATEGIES

# Datasets a testar
DATASETS_TO_TEST = list(DATASETS.keys())

logger.info(f"Modelos: {len(MODELS_TO_TEST)}")
logger.info(f"Estratégias: {len(STRATEGIES_TO_TEST)}")
logger.info(f"Datasets: {len(DATASETS_TO_TEST)}")
logger.info(f"Total de experimentos: {len(MODELS_TO_TEST) * len(STRATEGIES_TO_TEST) * len(DATASETS_TO_TEST)}")

## 2. Função de Execução de Experimento

In [ ]:
def extract_prediction(response: str) -> int:
    """Extrai predição da resposta do modelo."""
    response_lower = response.lower().strip()
    
    fake_keywords = ["falsa", "fake", "falso", "mentira", "desinformação", "1"]
    true_keywords = ["verdadeira", "verdadeiro", "real", "verídica", "0"]
    
    for keyword in fake_keywords:
        if keyword in response_lower:
            return 1
    
    for keyword in true_keywords:
        if keyword in response_lower:
            return 0
    
    return -1


def run_experiment(model_name, strategy, dataset_name, sample_size, seed):
    """Executa um único experimento."""
    logger.info(f"Experimento: {model_name} | {strategy} | {dataset_name}")
    
    try:
        # 1. Carregar dados
        logger.info("[1/4] Carregando dataset...")
        df = load_dataset(dataset_name)
        test_df = prepare_test_set(df, sample_size=sample_size, seed=seed)
        logger.info(f"      Amostras: {len(test_df)}")
        
        # 2. Carregar modelo (Ollama)
        logger.info("[2/4] Carregando modelo via Ollama...")
        model_handler = ModelHandlerOllama(model_name)
        vram_usage = model_handler.get_vram_usage()
        logger.info(f"      Modelo Ollama: {model_handler.ollama_model}")
        logger.info(f"      Memória estimada: {vram_usage:.2f} GB")
        
        # 3. Preparar prompts
        logger.info("[3/4] Preparando prompts...")
        prompt_builder = PromptBuilder(strategy)
        
        if strategy == "few_shot":
            examples_df = df[~df.index.isin(test_df.index)].sample(
                n=EXPERIMENT_CONFIG["few_shot_examples"],
                random_state=seed
            )
            prompt_builder.set_examples(examples_df)
        
        # 4. Executar inferência
        logger.info("[4/4] Executando inferência...")
        predictions = []
        inference_times = []
        
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="      "):
            prompt = prompt_builder.build_prompt(row["text"])
            
            start_time = time.time()
            response = model_handler.generate(prompt)
            end_time = time.time()
            
            prediction = extract_prediction(response)
            predictions.append(prediction)
            inference_times.append(end_time - start_time)
        
        # Calcular métricas
        true_labels = test_df["label"].tolist()
        metrics = calculate_metrics(true_labels, predictions)
        
        metrics["avg_inference_time"] = sum(inference_times) / len(inference_times)
        metrics["total_inference_time"] = sum(inference_times)
        metrics["vram_usage_gb"] = vram_usage
        
        # Resultados
        results = {
            "model": model_name,
            "strategy": strategy,
            "dataset": dataset_name,
            "sample_size": sample_size,
            "timestamp": datetime.now().isoformat(),
            "metrics": metrics,
        }
        
        # Liberar memória (Ollama gerencia automaticamente)
        model_handler.unload()
        
        logger.info(f"✅ Concluído! F1-Score: {metrics['f1_score']:.4f}")
        
        return results
        
    except Exception as e:
        logger.error(f"❌ ERRO: {e}")
        return {
            "model": model_name,
            "strategy": strategy,
            "dataset": dataset_name,
            "error": str(e)
        }

## 3. Executar Benchmark

In [ ]:
# Executar todos os experimentos
all_results = []
results_dir = Path(PATHS['results_dir'])
results_dir.mkdir(parents=True, exist_ok=True)

total_experiments = len(MODELS_TO_TEST) * len(STRATEGIES_TO_TEST) * len(DATASETS_TO_TEST)
experiment_count = 0

logger.info(f"Iniciando benchmark com {total_experiments} experimentos...")

for model_name in MODELS_TO_TEST:
    for strategy in STRATEGIES_TO_TEST:
        for dataset_name in DATASETS_TO_TEST:
            experiment_count += 1
            
            filename = f"{model_name.replace('/', '_')}_{strategy}_{dataset_name}.json"
            filepath = results_dir / filename
            
            # Verificar se já existe resultado de sucesso
            if filepath.exists():
                try:
                    with open(filepath, "r") as f:
                        existing_results = json.load(f)
                        
                    if "error" not in existing_results:
                        logger.info(f"[Experimento {experiment_count}/{total_experiments}] Pular: {model_name} | {strategy} | {dataset_name} (Já concluído)")
                        all_results.append(existing_results)
                        continue
                    else:
                        logger.info(f"[Experimento {experiment_count}/{total_experiments}] Re-executando (erro anterior): {model_name} | {strategy} | {dataset_name}")
                except (json.JSONDecodeError, Exception):
                    logger.info(f"[Experimento {experiment_count}/{total_experiments}] Re-executando (arquivo corrompido): {model_name} | {strategy} | {dataset_name}")

            logger.info(f"[Experimento {experiment_count}/{total_experiments}] Executando: {model_name} | {strategy} | {dataset_name}")
            
            results = run_experiment(
                model_name=model_name,
                strategy=strategy,
                dataset_name=dataset_name,
                sample_size=SAMPLE_SIZE,
                seed=SEED
            )
            
            all_results.append(results)
            
            # Salvar resultados parciais
            with open(filepath, "w") as f:
                json.dump(results, f, indent=2)

logger.info("✅ Benchmark concluído!")

## 4. Conclusão

✅ **Benchmark concluído!**

Os resultados foram salvos em arquivos JSON no diretório `reports/results/`.

Para analisar os resultados, execute o notebook `04_benchmark_analysis.ipynb`.

In [ ]:
# Resumo dos resultados salvos
logger.info(f"✅ Todos os resultados foram salvos em: {results_dir}")
logger.info(f"   Total de arquivos: {len(list(results_dir.glob('*.json')))}")
logger.info("   Execute o notebook 04_benchmark_analysis.ipynb para analisar os resultados.")